In [0]:
from google.colab import files

In [0]:
import numpy as np
import pandas as pd

In [0]:
# f = files.upload()

In [0]:
movies = pd.read_csv('movies.csv',encoding='latin1')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv',encoding='latin1')

In [219]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [220]:
ratings.head()

,userId,movieId,rating,timestamp
0,12882,1,4.0,1147195252
1,12882,32,3.5,1147195307
2,12882,47,5.0,1147195343
3,12882,50,5.0,1147185499
4,12882,110,4.5,1147195239


In [221]:
tags.head()

,movieId,userId,tag,timestamp
0,3916,12882,sports,1147195545
1,4085,12882,Eddie Murphy,1147195966
2,33660,12882,boxing,1147195514
3,1197,320,must show,1145964801
4,1396,320,must show,1145964810


In [222]:
#computing mean ratings of every user
Mean = ratings.groupby(by="userId", as_index=False)['rating'].mean()
ratings_new = pd.merge(ratings,Mean,on='userId')
ratings_new.head()

,userId,movieId,rating_x,timestamp,rating_y
0,12882,1,4.0,1147195252,4.061321
1,12882,32,3.5,1147195307,4.061321
2,12882,47,5.0,1147195343,4.061321
3,12882,50,5.0,1147185499,4.061321
4,12882,110,4.5,1147195239,4.061321


In [223]:
#predicting the number of movies the user has rated and the number of movies which have got higher number of ratings
ratings['usr_size']=None
ratings['mov_size']=None

#removing the movies having less than 50 ratings and number of users who have rated less than 50 movies
ratings['usr_size']=ratings[ratings['rating']>=0.0].groupby('userId').transform(np.size)
ratings['mov_size']=ratings[ratings['rating']>=0.0].groupby('movieId').transform(np.size)
pop_movs = ratings[(ratings['mov_size']>=50) & (ratings['usr_size']>=50)]
display(pop_movs)

,userId,movieId,rating,timestamp,usr_size,mov_size
0,12882,1,4.0,1147195252,106,496
1,12882,32,3.5,1147195307,106,424
2,12882,47,5.0,1147195343,106,452
3,12882,50,5.0,1147185499,106,454
4,12882,110,4.5,1147195239,106,477
...,...,...,...,...,...,...
264500,90097,54286,5.0,1336143258,82,287
264501,90097,60069,4.5,1336144698,82,335
264502,90097,68358,5.0,1336144031,82,263
264503,90097,68954,5.0,1336146596,82,271


In [224]:
#making pivot table for users and movies
user_movie_mat = pop_movs.pivot(index='userId', columns='movieId',values='rating').fillna(0)
display(user_movie_mat.head())

movieId,1,2,3,5,6,7,10,11,16,17,18,19,21,22,23,24,25,29,31,32,34,36,39,44,47,48,50,52,58,62,65,69,70,88,95,101,104,105,110,111,...,88140,88163,88744,88810,89492,89745,89864,90866,91500,91529,91542,91630,91658,92259,93840,94864,94959,95510,96079,96610,97304,97752,97913,97921,97938,98809,99114,102125,102445,104841,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,2.5,0.0,0.0,0.0,0.0,2.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
359,5.0,0.0,0.0,0.0,5.0,0.0,4.0,4.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0,3.5,0.0,2.0,0.0,3.0,4.0,3.0,0.0,1.5,4.0,0.0,4.5,0.0,0.0,3.5,0.0,0.0,3.5,0.0,3.0,5.0,0.0,0.0,3.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
370,4.5,4.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,...,3.0,4.0,0.0,0.0,4.0,2.5,0.0,0.0,3.5,0.0,0.0,0.0,0.0,2.5,4.5,1.0,0.0,0.0,4.0,4.0,0.0,0.0,3.5,4.0,0.0,4.0,4.5,1.5,0.0,0.0,2.5,3.0,4.5,4.0,0.0,0.0,3.0,4.5,3.5,3.0
910,5.0,4.0,3.5,3.5,3.5,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,4.5,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,4.0,5.0,...,3.5,2.0,0.0,4.5,4.5,0.0,0.0,4.0,0.0,0.0,3.5,0.0,4.0,0.0,0.0,0.0,0.0,3.5,3.5,2.0,3.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,3.5,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,4.5,0.0,0.0


In [0]:
from scipy.sparse import csr_matrix

# we use sparse matrix to prevent overflow due to large number of zeroes
sparse_usermovie = csr_matrix(user_movie_mat)

In [226]:
from sklearn.neighbors import NearestNeighbors

# fitting on knn model
model = NearestNeighbors(n_neighbors=20, metric='cosine', algorithm='brute',n_jobs=-1)
model.fit(sparse_usermovie)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=20, p=2,
                 radius=1.0)

In [0]:
num_movies = len(movies.movieId.unique())

# making a mapper for mapping movies and id
movies_list = movies['title']
nums = [int(x) for x in range(0,num_movies)]

zipobj = zip(movies_list, nums)
movie_to_idx= dict(zipobj)

In [0]:
# !pip install fuzzywuzzy
from fuzzywuzzy import fuzz

In [0]:
def make_recommendations(model_knn, data, mapper, fav_movie, n_recommendations):
    # fit
    model_knn.fit(data)
    # get input movie index
    print('input:', fav_movie)
    idx = similarity(mapper, fav_movie, verbose=True)
    
    distances, indices = model_knn.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    
    raw_recommends =  sorted(zip(indices.squeeze().tolist(), distances.squeeze().tolist()), key=lambda x: x[1])[:0:-1]
    # get reverse mapper
    reverse_mapper = {v: k for k, v in mapper.items()}
    # print recommendations
    print('Recommendations for {}:'.format(fav_movie))
    for i, (idx, dist) in enumerate(raw_recommends):
        print('{0}: {1}, {2}'.format(i+1, reverse_mapper[idx], dist))

In [0]:
def similarity(mapper, fav_movie, verbose=True):
   
    match_tuple = []
    # get match
    for title, idx in mapper.items():
        ratio = fuzz.ratio(title.lower(), fav_movie.lower())
        if ratio >= 60:
            match_tuple.append((title, idx, ratio))
    # sort
    match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
    if not match_tuple:
        print('Oops! No match is found')
        return
    if verbose:
        print('Possible recommendations: {0}\n'.format([x[0] for x in match_tuple]))
    return match_tuple[0][1]


In [231]:
user_input = 'jumanji'

# making recommendations for jumanji
make_recommendations(
    model_knn=model,
    data=sparse_usermovie,
    fav_movie=user_input,
    mapper=movie_to_idx,
    n_recommendations=5)

input: jumanji
Possible recommendations: ['Jumanji (1995)']

Recommendations for jumanji:
1: Free Willy 2: The Adventure Home (1995), 0.6242835138766227
2: Crying Game, The (1992), 0.6231949889414417
3: Bride of Frankenstein, The (Bride of Frankenstein) (1935), 0.6082168603867244
4: Richie Rich (1994), 0.5949544625690844
5: Stalker (1979), 0.5687955284695971


In [233]:
# making a pivot and filling NA values with average movie rating
avg_umt = pop_movs.pivot(index='userId', columns='movieId',values='rating')
avg_umt = avg_umt.fillna(avg_umt.mean(axis=0))
display(avg_umt)

movieId,1,2,3,5,6,7,10,11,16,17,18,19,21,22,23,24,25,29,31,32,34,36,39,44,47,48,50,52,58,62,65,69,70,88,95,101,104,105,110,111,...,88140,88163,88744,88810,89492,89745,89864,90866,91500,91529,91542,91630,91658,92259,93840,94864,94959,95510,96079,96610,97304,97752,97913,97921,97938,98809,99114,102125,102445,104841,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,2.500000,3.070144,2.94,2.848684,3.818182,2.000000,2.50000,3.391509,3.901596,3.908,3.309859,2.681818,3.402439,3.056452,3.039216,2.80303,3.643939,3.810811,3.136364,3.000000,2.000000,3.710526,3.205882,2.622549,3.996606,2.915385,4.171558,3.424528,3.514706,3.491228,2.153846,3.44,3.335294,3.133333,2.791139,3.640244,3.406542,3.212963,3.72807,3.931227,...,3.465,3.758621,3.522523,3.83871,3.945783,3.952381,3.888889,3.636986,3.632143,3.815152,3.676471,3.583333,3.714286,4.137097,3.716867,3.234375,3.931818,3.5,3.768908,3.666667,3.838384,3.671429,3.860294,3.821429,3.772152,3.601626,4.056738,3.569149,3.651042,3.851351,3.689189,3.595238,3.863636,3.811688,4.006173,3.927536,3.835938,3.851351,4.0,3.97973
320,3.790123,3.070144,2.94,2.848684,3.818182,3.228395,3.34127,3.391509,3.901596,3.908,3.309859,2.681818,3.402439,3.056452,3.039216,2.80303,3.643939,3.810811,3.136364,3.966346,3.275281,3.710526,3.205882,2.622549,4.000000,2.915385,5.000000,3.424528,3.514706,3.491228,2.153846,3.44,3.335294,3.133333,2.791139,3.640244,3.406542,3.212963,3.72807,3.931227,...,3.465,3.758621,3.522523,3.83871,3.945783,3.952381,3.888889,3.636986,3.632143,3.815152,3.676471,3.583333,3.714286,4.137097,3.716867,3.234375,3.931818,3.5,3.768908,3.666667,3.838384,3.671429,3.860294,3.821429,3.772152,3.601626,4.056738,3.569149,3.651042,3.851351,3.689189,3.595238,3.863636,3.811688,4.006173,3.927536,3.835938,3.851351,4.0,3.97973
359,5.000000,3.070144,2.94,2.848684,5.000000,3.228395,4.00000,4.000000,3.901596,3.908,3.309859,3.000000,3.402439,4.000000,3.039216,3.50000,3.643939,2.000000,3.136364,3.000000,4.000000,3.000000,3.205882,1.500000,4.000000,2.915385,4.500000,3.424528,3.514706,3.500000,2.153846,3.44,3.500000,3.133333,3.000000,5.000000,3.406542,3.212963,3.50000,3.931227,...,3.465,3.758621,3.522523,3.83871,3.945783,3.952381,3.888889,3.636986,3.632143,3.815152,3.676471,3.583333,3.714286,4.137097,3.716867,3.234375,3.931818,3.5,3.768908,3.666667,3.838384,3.671429,3.860294,3.821429,3.772152,3.601626,4.056738,3.569149,3.651042,3.851351,3.689189,3.595238,3.863636,3.811688,4.006173,3.927536,3.835938,3.851351,4.0,3.97973
370,4.500000,4.000000,2.94,2.848684,5.000000,3.228395,3.34127,3.391509,3.901596,3.908,3.309859,2.681818,3.402439,3.056452,3.039216,2.80303,4.500000,3.810811,3.136364,5.000000,3.275281,3.710526,3.205882,2.622549,5.000000,2.915385,4.000000,3.424528,3.514706,3.491228,2.153846,3.44,4.000000,3.133333,2.791139,3.640244,3.406542,3.212963,3.72807,4.500000,...,3.000,4.000000,3.522523,3.83871,4.000000,2.500000,3.888889,3.636986,3.500000,3.815152,3.676471,3.583333,3.714286,2.500000,4.500000,1.000000,3.931818,3.5,4.000000,4.000000,3.838384,3.671429,3.500000,4.000000,3.772152,4.000000,4.500000,1.500000,3.651042,3.851351,2.500000,3.000000,4.500000,4.000000,4.006173,3.927536,3.000000,4.500000,3.5,3.00000
910,5.000000,4.000000,3.50,3.500000,3.500000,3.228395,3.34127,4.000000,4.000000,3.908,3.309859,2.681818,3.402439,3.056452,3.039216,2.80303,3.000000,3.810811,3.136364,4.000000,3.000000,3.710526,3.205882,2.622549,3.996606,2.915385,4.171558,4.500000,3.514706,4.500000,2.153846,3.44,3.335294,3.133333,3.500000,3.640244,3.406542,3.212963,4.00000,5.000000,...,3.500,2.000000,3.522523,4.50000,4.500000,3.952381,3.888889,4.000000,3.632143,3.815152,3.500000,3.583333,4.000000,4.137097,3.716867,3.234375,3.931818,3.5,3.500000,2.000000,3.000000,3.671429,3.860294,4.000000,3.772152,3.000000,4.056738,3.569149,3.500000,3.851351,3.689189,3.595238,3.500000,3.811688,4.006173,3.927536,3.835938,4.500000,4.0,3.97973
...,...,...,...,...,...,...,...,.

In [234]:
from sklearn.metrics.pairwise import cosine_similarity

#cosine similarity for getting similarity ratio between two users
cos_sim = cosine_similarity(avg_umt)
np.fill_diagonal(cos_sim, 0)
similarity = pd.DataFrame(cos_sim, index=avg_umt.index)
similarity.columns = avg_umt.index
similarity.head()

userId,316,320,359,370,910,975,1015,1387,1588,1738,1741,1835,1935,2024,2248,2702,2847,2945,2988,3029,3076,3569,3665,3915,4215,4347,4483,4629,4931,5063,5450,5475,5611,6116,6937,6988,7541,7723,7880,8527,...,130955,131331,131351,131439,131616,131620,131630,131910,132187,132229,132295,132470,132595,132914,133047,133811,134156,134181,134279,134316,134401,134521,134525,134557,134627,134964,135092,135223,135418,136015,136455,136856,137118,137209,137227,137446,137559,137805,138072,138200
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,0.000000,0.997767,0.987902,0.985551,0.988378,0.997116,0.993964,0.995586,0.989464,0.996254,0.981839,0.994950,0.991100,0.994826,0.996574,0.995688,0.993529,0.992002,0.985000,0.991224,0.996232,0.996857,0.993575,0.994796,0.991648,0.980236,0.990808,0.996359,0.995495,0.985457,0.992844,0.988408,0.996761,0.995400,0.995644,0.988699,0.997566,0.993745,0.992791,0.996211,...,0.997731,0.997532,0.996758,0.990624,0.997669,0.987084,0.990409,0.988303,0.984658,0.997222,0.993880,0.995170,0.993263,0.989954,0.992525,0.983480,0.987958,0.991247,0.992747,0.997713,0.974491,0.981471,0.993119,0.996216,0.994502,0.995794,0.989579,0.986694,0.990419,0.987164,0.993852,0.996125,0.993878,0.996586,0.994200,0.997466,0.997188,0.969193,0.976725,0.991946
320,0.997767,0.000000,0.988994,0.986944,0.989150,0.997845,0.994657,0.996579,0.990601,0.997494,0.982582,0.996545,0.991779,0.995702,0.997391,0.996709,0.994568,0.993166,0.985458,0.992216,0.996749,0.997655,0.995005,0.995429,0.992936,0.981275,0.992178,0.996908,0.996478,0.986617,0.992906,0.990020,0.997827,0.996726,0.996660,0.989165,0.998697,0.995077,0.993766,0.997550,...,0.998569,0.998330,0.997701,0.991021,0.998602,0.987745,0.991145,0.989091,0.986081,0.998185,0.995026,0.995844,0.994845,0.991013,0.993443,0.985004,0.988502,0.992770,0.993499,0.998646,0.975305,0.982263,0.994080,0.997125,0.995813,0.996715,0.990331,0.987448,0.991174,0.988713,0.994731,0.997071,0.995383,0.996950,0.995013,0.998358,0.998173,0.970387,0.977538,0.992840
359,0.987902,0.988994,0.000000,0.978862,0.980412,0.987787,0.985411,0.986664,0.981775,0.988122,0.973140,0.986555,0.982794,0.985235,0.987887,0.987608,0.984798,0.983392,0.975348,0.982995,0.987479,0.988195,0.984609,0.986291,0.983563,0.971190,0.981491,0.986894,0.986050,0.976889,0.982996,0.979447,0.987695,0.987102,0.987177,0.979501,0.988464,0.986003,0.984680,0.987668,...,0.988728,0.989045,0.987375,0.980811,0.989180,0.979171,0.980964,0.979238,0.977218,0.988557,0.983932,0.986047,0.984012,0.981685,0.983364,0.977549,0.979231,0.983899,0.984513,0.989126,0.963301,0.970693,0.984254,0.986776,0.985821,0.986716,0.981413,0.977075,0.981069,0.977618,0.985388,0.987216,0.985348,0.987143,0.985744,0.988777,0.988862,0.957867,0.965470,0.982447
370,0.985551,0.986944,0.978862,0.000000,0.978548,0.985937,0.983608,0.984368,0.979931,0.986525,0.969906,0.984706,0.979472,0.984629,0.985977,0.985415,0.982388,0.980950,0.975950,0.980924,0.984389,0.985952,0.983164,0.983886,0.981746,0.968935,0.979645,0.985036,0.983748,0.973914,0.981519,0.979106,0.986543,0.985723,0.985278,0.976093,0.987104,0.983896,0.982052,0.985948,...,0.987014,0.986713,0.986161,0.978288,0.986755,0.976811,0.979428,0.978736,0.973918,0.986587,0.981004,0.984183,0.983080,0.979032,0.978902,0.972872,0.976205,0.980816,0.982940,0.987060,0.961398,0.969191,0.981970,0.985218,0.984243,0.984821,0.978291,0.974319,0.978484,0.977392,0.982276,0.985428,0.984669,0.985487,0.983620,0.986570,0.986998,0.956123,0.965183,0.980788
910,0.988378,0.989150,0.980412,0.978548,0.000000,0.988235,0.986158,0.986618,0.980949,0.987555,0.972504,0.987036,0.981786,0.985702,0.988200,0.987131,0.985088,0.983426,0.975487,0.984651,0.987386,0.988014,0.984982,0.985898,0.984880,0.971368,0.982581,0.987289,0.986626,0.977633,0.983324,0.979976,0.988876,0.987741,0.987832,0.978488,0.989122,0.986557,0.984819,0.987875,...,0.988972,0.989361,0.987881,0.980055,0.988984,0.980256,0.979977,0.979523,0.978449,0.988731,0.984855,0.986525,0.985840,0.982116,0.983099,0.978308,0.9

In [0]:
#function to get top N neighbours
def getNeighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [236]:
# finding top 10 similar users for every user
recomm = getNeighbours(similarity, 10)
recomm.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10
userId,,,,,,,,,,
316,22338,121386,124936,88455,57474,14833,103927,68702,96864,94676
320,22338,121386,124936,88455,57474,130671,103927,114448,56392,119625
359,124936,22338,57474,17039,107991,121386,88455,14833,75001,131616
370,17039,22338,123707,88455,101004,42367,56392,100728,120308,57474
910,107991,22338,88455,124936,94676,103927,17039,92106,43604,134316


In [0]:
# finding similar movies between two users by combining their watched movies
def common_movs(u1, u2):
  user1 = pop_movs.loc[pop_movs['userId']==u1,'movieId']
  user2 = pop_movs.loc[pop_movs['userId']==u2,'movieId']
  common = pd.merge(user1, user2, how='inner', on='movieId').values.tolist()
  movs=[]
  for i in range(len(common)):
    movs.append(movies.loc[movies['movieId']==common[i][0],'title'])
  return movs

In [238]:
# common movies shared
print("common movies shared by these users are:\n")
print(common_movs(12882,90097))

common movies shared by these users are:

[42    Usual Suspects, The (1995)
Name: title, dtype: object, 76    Braveheart (1995)
Name: title, dtype: object, 89    Apollo 13 (1995)
Name: title, dtype: object, 157    Star Wars: Episode IV - A New Hope (1977)
Name: title, dtype: object, 189    Shawshank Redemption, The (1994)
Name: title, dtype: object, 213    Forrest Gump (1994)
Name: title, dtype: object, 839    Good Will Hunting (1997)
Name: title, dtype: object, 964    Saving Private Ryan (1998)
Name: title, dtype: object, 1065    Few Good Men, A (1992)
Name: title, dtype: object, 1210    Matrix, The (1999)
Name: title, dtype: object, 1227    Star Wars: Episode I - The Phantom Menace (1999)
Name: title, dtype: object, 1846    Ocean's Eleven (2001)
Name: title, dtype: object, 1854    Lord of the Rings: The Fellowship of the Ring,...
Name: title, dtype: object, 1907    Bourne Identity, The (2002)
Name: title, dtype: object, 1959    Lord of the Rings: The Two Towers, The (2002)
Name: titl